### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [5]:
from accounts import Account

In [22]:
account = Account.get("Urjashee")
account

Account(name='urjashee', balance=9921.844, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 26.052 each.], portfolio_value_time_series=[('2025-09-02 04:09:52', 10119.844), ('2025-09-02 04:09:53', 10059.844)])

In [23]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "urjashee", "balance": 9684.369999999999, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 26.052, "timestamp": "2025-09-02 04:09:52", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 79.158, "timestamp": "2025-09-02 04:10:20", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-09-02 04:09:52", 10119.844], ["2025-09-02 04:09:53", 10059.844], ["2025-09-02 04:10:20", 10266.369999999999]], "total_portfolio_value": 10266.369999999999, "total_profit_loss": 266.3700000000008}'

In [24]:
account.report()

'{"name": "urjashee", "balance": 9684.369999999999, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 26.052, "timestamp": "2025-09-02 04:09:52", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 79.158, "timestamp": "2025-09-02 04:10:20", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-09-02 04:09:52", 10119.844], ["2025-09-02 04:09:53", 10059.844], ["2025-09-02 04:10:20", 10266.369999999999], ["2025-09-02 04:10:21", 10074.369999999999]], "total_portfolio_value": 10074.369999999999, "total_profit_loss": 74.3700000000008}'

In [25]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 26.052,
  'timestamp': '2025-09-02 04:09:52',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 79.158,
  'timestamp': '2025-09-02 04:10:20',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [20]:
params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

In [21]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, annotations=None, meta=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, annotations=None, meta=None),
 Tool(name='buy_shares', title=None, description=

In [26]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Urjashee and my account is under the name Urjashee. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [27]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Urjashee, your current cash balance is approximately $9,684.37. In terms of holdings, you have 6 shares of AMZN (Amazon). Is there anything specific you would like to do with your account?

### Now let's build our own MCP Client

In [28]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='buy_shares', title=None, description="B

In [29]:
request = "My name is Urjashee and my account is under the name Urjashee. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your current account balance is $9,684.37. If you have any other questions or want to make transactions, just let me know!

In [30]:
context = await read_accounts_resource("Urjashee")
print(context)

{"name": "urjashee", "balance": 9684.369999999999, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 26.052, "timestamp": "2025-09-02 04:09:52", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 79.158, "timestamp": "2025-09-02 04:10:20", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-09-02 04:09:52", 10119.844], ["2025-09-02 04:09:53", 10059.844], ["2025-09-02 04:10:20", 10266.369999999999], ["2025-09-02 04:10:21", 10074.369999999999], ["2025-09-02 04:18:45", 9894.369999999999]], "total_portfolio_value": 9894.369999999999, "total_profit_loss": -105.6299999999992}


In [31]:
from accounts import Account
Account.get("Urjashee").report()

'{"name": "urjashee", "balance": 9684.369999999999, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 26.052, "timestamp": "2025-09-02 04:09:52", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 79.158, "timestamp": "2025-09-02 04:10:20", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-09-02 04:09:52", 10119.844], ["2025-09-02 04:09:53", 10059.844], ["2025-09-02 04:10:20", 10266.369999999999], ["2025-09-02 04:10:21", 10074.369999999999], ["2025-09-02 04:18:45", 9894.369999999999], ["2025-09-02 04:19:05", 9954.369999999999]], "total_portfolio_value": 9954.369999999999, "total_profit_loss": -45.6299999999992}'